# Energy level calculator
Calculating energies for given states at zero electric and magnetic field.

## Import packages

In [1]:
%load_ext autoreload
%autoreload 2

import copy
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import scipy

from centrex_TlF_hamiltonian.states import State, CoupledBasisState, UncoupledBasisState, generate_uncoupled_states_ground_mF0, generate_coupled_states_excited_mF0, find_state_idx_from_state, ElectronicState
from centrex_TlF_hamiltonian.hamiltonian import generate_uncoupled_hamiltonian_X, generate_uncoupled_hamiltonian_X_function, generate_coupled_hamiltonian_B, generate_coupled_hamiltonian_B_function, XConstants, BConstants
from centrex_TlF_hamiltonian.hamiltonian.utils import matrix_to_states

## Tl-205
### X-state Hamiltonian

In [2]:
# Generate quantum numbers
QN_X_uc = generate_uncoupled_states_ground_mF0(range(0,12))

# Generate Hamiltonian
H_X = generate_uncoupled_hamiltonian_X_function(generate_uncoupled_hamiltonian_X(QN_X_uc))(np.array([0,0,0]),np.array([0,0,1e-4]))/(2*np.pi)

#Diagonalize the Hamiltonian and define a new basis where the matrix is diagonal
D, V = np.linalg.eigh(H_X)
H_X_diag = V.conj().T @ H_X @ V

#Remove very small entries from V
V[np.abs(V) < 1e-5] = 0

#New set of quantum numbers:
QN_X_diag = matrix_to_states(V, QN_X_uc)

### B-state Hamiltonian

In [3]:
# Generate quantum numbers
QN_B_c = generate_coupled_states_excited_mF0(range(1,13), Ps = [-1,1])

# Generate Hamiltonian
H_B = generate_coupled_hamiltonian_B_function(generate_coupled_hamiltonian_B(QN_B_c))(np.array([0,0,0]),np.array([0,0,1e-4]))/(2*np.pi)

#Diagonalize the Hamiltonian and define a new basis where the matrix is diagonal
D, V = np.linalg.eigh(H_B)
H_B_diag = V.conj().T @ H_B @ V

#Remove very small entries from V
V[np.abs(V) < 1e-5] = 0

#New set of quantum numbers:
QN_B_diag = matrix_to_states(V, QN_B_c)

### Merge Hamiltonians
Merge the X- and B-state Hamiltonians and add and offset to the B-state energies to include the optical wavelength

In [4]:
#Add an offset to the energies of the B-states so that the transition frequency from |X, J = 0, F=1> to |B, J = 1, F = 2> is 
#1103407.96 GHz
X_state = 1*CoupledBasisState(J=0, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, electronic_state=ElectronicState.X, P = +1, Omega = 0)
B_state = 1*CoupledBasisState(J=1, F1 = 3/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, electronic_state=ElectronicState.B, P = -1, Omega = 1)

#Find the indices that correspond to each state
i_X = find_state_idx_from_state(H_X_diag, X_state, QN_X_diag)
i_B = find_state_idx_from_state(H_B_diag, B_state, QN_B_diag)

#Find the correct energy shift
E_shift = 4*275851.99e9- H_B_diag[i_B,i_B] + H_X_diag[i_X,i_X] 

#Shift the energies of the B-states
H_B_diag = H_B_diag + E_shift*np.eye(H_B_diag.shape[0])

# Make the total Hamiltonian for Tl-205
H_tot = scipy.linalg.block_diag(H_X_diag, H_B_diag)
QN = QN_X_diag + QN_B_diag

## Calculate energy differences

In [5]:
def calculate_transition_frequency(state1, state2, H, QN):
    #Find indices corresponding to each state
    i1 = find_state_idx_from_state(H, state1, QN)
    i2 = find_state_idx_from_state(H, state2, QN)
    
#     print(i1)
#     print(i2)
    
    #Find energies of each state
    E1 = H[i1,i1]
    E2 = H[i2,i2]
    
    #Calculate transition frequency
    f = E2 - E1
    
    return np.real(f)

## Spacing of rotational levels
calculating the spacings of rotational levels to within 10 MHz

In [9]:
Js = [0,1,2,3,4]
for J in Js:
    state1 = 1*CoupledBasisState(J=J, F1 = J+1/2, F = J+1, mF = 0, I1 =1/2, I2 = 1/2, electronic_state=ElectronicState.X, P = (-1)**(J), Omega = 0)
    state2 = 1*CoupledBasisState(J=J+1, F1 = J+3/2, F = J+2, mF = 0, I1 =1/2, I2 = 1/2, electronic_state=ElectronicState.X, P = (-1)**(J+1), Omega = 0)
    freq = calculate_transition_frequency(state1, state2, H_tot, QN)
    print(f"Energy spacing between J={J} and J={J+1}: {freq/1e9:.2f} GHz")

Energy spacing between J=0 and J=1: 13.33 GHz
Energy spacing between J=1 and J=2: 26.67 GHz
Energy spacing between J=2 and J=3: 40.00 GHz
Energy spacing between J=3 and J=4: 53.34 GHz
Energy spacing between J=4 and J=5: 66.67 GHz


## Spacing of hyperfine levels

In [19]:
energies = {}
for J in Js:
    energies[J] = []
    print(f"Hyperfine spacings for J = {J}:")
    for F1 in np.arange(np.abs(J-1/2), J+1/2+1):
        for F in np.arange(np.abs(F1-1/2), F1+1/2+1):
            state = 1*CoupledBasisState(J=J, F1 = F1, F = F, mF = 0, I1 =1/2, I2 = 1/2, electronic_state=ElectronicState.X, P = (-1)**(J), Omega = 0)
            idx = find_state_idx_from_state(H_tot, state, QN)
            energies[J].append(H_tot[idx,idx])
    print(f"\t{np.diff(np.real(energies[J]))/1e3} kHz")

Hyperfine spacings for J = 0:
	[-13.30000015] kHz
Hyperfine spacings for J = 1:
	[ 22.23878262 175.95243475  14.53878252] kHz
Hyperfine spacings for J = 2:
	[ 44.51885069 278.81229832  35.21885079] kHz
Hyperfine spacings for J = 3:
	[ 63.94494862 384.58010254  54.37828195] kHz
Hyperfine spacings for J = 4:
	[ 82.65417213 491.38165617  72.9905354 ] kHz
